# First Try

In [4]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
data = pd.read_csv('data\\train.csv')